#### Importing Lib.

In [82]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K

#### Loading the CSV files and preparing a vocabulary

In [83]:
# Paths
dataset_path = '/kaggle/input/mathink-dataset-1/dataset_1'
images_path = os.path.join(dataset_path, 'processed_img')

# Load CSV files
train_df = pd.read_csv(os.path.join(dataset_path, 'im2latex_train.csv'))
valid_df = pd.read_csv(os.path.join(dataset_path, 'im2latex_validate.csv'))
test_df  = pd.read_csv(os.path.join(dataset_path, 'im2latex_test.csv'))

# Cleaning the test data as it has some NaN values
test_df = test_df.dropna(subset=['formula'])

# Build character vocabulary
all_texts = pd.concat([train_df['formula'], valid_df['formula'], test_df['formula']])
unique_chars = set(''.join(all_texts))

char_to_num = {char: idx for idx, char in enumerate(sorted(unique_chars))}
num_to_char = {idx: char for char, idx in char_to_num.items()}
vocab_size = len(char_to_num)

print(f"Vocabulary Size: {vocab_size}")
print(f"Example mapping: {list(char_to_num.items())[:10]}")

Vocabulary Size: 93
Example mapping: [(' ', 0), ('!', 1), ('"', 2), ('#', 3), ('&', 4), ("'", 5), ('(', 6), (')', 7), ('*', 8), ('+', 9)]


#### Defineing a custome data generator function with CTC inputs

In [84]:
class CTCDataGenerator(Sequence):
    def __init__(self, dataframe, images_dir, batch_size, img_height, img_width, datagen, max_label_length=100, shuffle=True):
        self.dataframe = dataframe
        self.images_dir = images_dir
        self.batch_size = batch_size
        self.img_height = img_height
        self.img_width = img_width
        self.datagen = datagen
        self.shuffle = shuffle
        self.indices = np.arange(len(self.dataframe))
        self.max_label_length = max_label_length
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def encode_label(self, text):
        return [char_to_num[char] for char in text]

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_filenames = self.dataframe.iloc[batch_indices]['image'].values
        batch_labels_text = self.dataframe.iloc[batch_indices]['formula'].values
        
        batch_images = []
        batch_labels = []
        label_lengths = []
    
        for filename, label_text in zip(batch_filenames, batch_labels_text):
            img_path = os.path.join(self.images_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (self.img_width, self.img_height))
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=-1)
            batch_images.append(img)
            
            encoded_label = self.encode_label(label_text)
            batch_labels.append(encoded_label)
            label_lengths.append(len(encoded_label))
    
        batch_images = np.array(batch_images)
    
        batch_labels_padded = tf.keras.preprocessing.sequence.pad_sequences(
            batch_labels,
            maxlen=self.max_label_length,
            padding='post',
            value=len(char_to_num)
        )
    
        # Calculate input_length based on image width
        feature_width = self.img_width // 4  # After pooling layers, width is 48
        input_lengths = np.ones((len(batch_labels), 1), dtype='int32') * feature_width
        
        # Ensure label_lengths is reshaped correctly to (batch_size, 1)
        label_lengths = np.array(label_lengths).reshape(-1, 1)
    
        inputs = {
            'input_image': batch_images,
            'labels': batch_labels_padded,
            'input_length': input_lengths,
            'label_length': label_lengths
        }
        outputs = {'ctc': np.zeros((len(batch_images), 1))}
    
        return inputs, outputs


#### Setting up data augmentation

In [85]:
# Augmentation settings
train_datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

valid_datagen = ImageDataGenerator()

# Generator instances
batch_size = 32
img_height = 192
img_width = 192

train_generator = CTCDataGenerator(train_df, images_path, batch_size, img_height, img_width, train_datagen)
valid_generator = CTCDataGenerator(valid_df, images_path, batch_size, img_height, img_width, valid_datagen)


#### Building the CRNN model with CTC Loss

In [86]:
# Define Inputs
input_image = layers.Input(shape=(img_height, img_width, 1), name='input_image')
labels = layers.Input(shape=(None,), dtype='int32', name='labels')
input_length = layers.Input(shape=(1,), dtype='int32', name='input_length')
label_length = layers.Input(shape=(1,), dtype='int32', name='label_length')

# CNN layers
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_image)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Reshape for RNN
new_shape = (img_width // 4, (img_height // 4) * 64)
x = layers.Reshape(target_shape=new_shape)(x)

# RNN
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)

# Dense output layer
x = layers.Dense(vocab_size + 1, activation='softmax')(x)  # vocab_size + blank token

# CTC Loss Layer
def ctc_lambda_func(inputs):
    labels, y_pred, input_length, label_length = inputs
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

ctc_loss = layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
    [labels, x, input_length, label_length]
)

# Define Model
model = models.Model(inputs=[input_image, labels, input_length, label_length], outputs=ctc_loss)

# Compile Model
model.compile(optimizer='adam')
model.summary()




Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)  │ (None, 192, 192, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_24 (Conv2D)        │ (None, 192, 192, 32)   │            320 │ input_image[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_22          │ (None, 96, 96, 32)     │              0 │ conv2d_24[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_25 (Conv2D)        │ (None, 96, 96, 64)     │         18,496 │ max_pooling2d_22[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_23          │ (None, 48, 48, 64)     │              0 │ conv2d_25[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_11 (Reshape)      │ (None, 48, 3072)       │              0 │ max_pooling2d_23[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_11          │ (None, 48, 256)        │      3,277,824 │ reshape_11[0][0]       │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ labels (InputLayer)       │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 48, 94)         │         24,158 │ bidirectional_11[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_length (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ label_length (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ctc (Lambda)              │ (None, 1)              │              0 │ labels[0][0],          │
│                           │                        │                │ dense_11[0][0],        │
│                           │                        │                │ input_length[0][0],    │
│                           │                        │                │ label_length[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,320,798 (12.67 MB)

 Trainable params: 3,320,798 (12.67 MB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# Train the model
model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30
)

Epoch 1/30


ValueError: Exception encountered when calling Lambda.call().

[1mCan not squeeze dim[1], expected a dimension of 1, got 100 for '{{node functional_11_1/ctc_1/Squeeze}} = Squeeze[T=DT_INT32, squeeze_dims=[-1]](data_1)' with input shapes: [?,100].[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(None, 1), dtype=int32)', 'tf.Tensor(shape=(None, 48, 94), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=int32)', 'tf.Tensor(shape=(None, 100), dtype=int32)']
  • mask=['None', 'None', 'None', 'None']
  • training=True